# Data Cleaning

The following are the features of the movie industry we will be exploring
along with the datasets that are needed to analyze them:

- Genre: `imdb.title.basics.csv.bz2`
- Runtime: `imdb.title.basics.csv.bz2`
- Budget Allocation: `tn.movie_budgets.csv.bz2`
- Release Window: `tn.movie_budgets.csv.bz2`, `tmdb.movies.csv.bz2`
- Director: `imdb.title.crew.csv.bz2`, `imdb.name.basics.csv.bz2`

Consolidated the list of datasets is:

- `tn.movie_budgets.csv.bz2`
- `tmdb.movies.csv.bz2`
- `imdb.title.crew.csv.bz2`
- `imdb.name.basics.csv.bz2`
- `imdb.title.basics.csv.bz2`

Our methodology will be as follows for each dataset:

1. Identify wrongly encoded data types
2. Impute/Drop missing values
3. Drop columns which aren't required

Finally we will try to unify our datasets into one csv file for simple loading.

## imports and functions

In [1]:
# filter out warnings
from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
# import our required libraries
import pandas as pd
import numpy as np
from src.tools import currency_string_to_float

In [3]:
# function to import dataset display info and head
def import_and_display(filepath: str, *args, **kwargs):
    """load a dataset display info and print head"""
    # load our dataset
    df = pd.read_csv(filepath, *args, **kwargs)
    # display info and head
    df.info()
    display(df.head())
    return df

## tn.movie_budgets.csv.bz2

In [4]:
# import our dataset, display info and head
tn_movie_budgets = import_and_display("../data/raw/tn.movie_budgets.csv.bz2")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


Our dataset doesn't look to have any issues, there are no known missing values, but we do have columns that should be numeric encoded as objects.

In [5]:
# convert release_date to a datetime instance
tn_movie_budgets.release_date = pd.to_datetime(tn_movie_budgets.release_date)

In [6]:
# convert our budget and gross columns to float
cols_to_convert = ['production_budget', 'domestic_gross', 'worldwide_gross']
result = tn_movie_budgets[cols_to_convert].applymap(currency_string_to_float)
tn_movie_budgets[cols_to_convert] = result

In [7]:
# finally drop the id column
tn_movie_budgets.drop(columns="id", errors="ignore", inplace=True)

## tmdb.movies.csv.bz2

In [8]:
# import our dataset, display info and head
tmdb_movies = import_and_display("../data/raw/tmdb.movies.csv.bz2")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


For this dataset we really only need to keep the release data, and a key to join on. We'll convert the release_date
to a datetime object and then drop the unnecessary columns.

In [9]:
# keep only original_title and title, and release date
keep_cols = ['original_title', 'title', 'release_date']
tmdb_movies = tmdb_movies[keep_cols]

In [10]:
# convert our datetime column
tmdb_movies.release_date = pd.to_datetime(tmdb_movies.release_date)

In [11]:
# are all original titles the same as title
title_change_movies = tmdb_movies.original_title != tmdb_movies.title
print(f"Movies with title change: {title_change_movies.sum()/tmdb_movies.shape[0]:.2%}")

Movies with title change: 9.59%


One way we can deal with this is by creating a new dataframe with only one title column and release date, and then removing duplicates.
This'll allow us to retain as much data as possible.

In [12]:
# combine our two title columns, exlode them
tmdb_movies["title_tup"] = tmdb_movies.apply(lambda row: (row[0], row[1]), axis=1)
tmdb_movies = tmdb_movies.explode("title_tup")[["title_tup", "release_date"]]
print(f"Exploded Size: {tmdb_movies.shape[0]:g}")

Exploded Size: 53034


In [13]:
# drop duplicates and rename columns
tmdb_movies.drop_duplicates(inplace=True)
tmdb_movies.rename(columns={"title_tup": "title"}, inplace=True)

## imdb.title.crew.csv.bz2

In [14]:
# load our dataset
imdb_title_crew = import_and_display("../data/raw/imdb.title.crew.csv.bz2")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   tconst     146144 non-null  object
 1   directors  140417 non-null  object
 2   writers    110261 non-null  object
dtypes: object(3)
memory usage: 3.3+ MB


,tconst,directors,writers
0,tt0285252,nm0899854,nm0899854
1,tt0438973,NaN,"nm0175726,nm1802864"
2,tt0462036,nm1940585,nm1940585
3,tt0835418,nm0151540,"nm0310087,nm0841532"
4,tt0878654,"nm0089502,nm2291498,nm2292011",nm0284943


This dataset is interesting it appears to be keys to be joined to the other imdb datasets. cleaning it up will be simple. We'll drop the writers column and explode the directors column.

In [15]:
# drop writers column, convert each directors to list of str
imdb_title_crew.drop(columns="writers", inplace=True, errors="ignore")
imdb_title_crew.directors = imdb_title_crew.directors.str.split(",")

In [16]:
# explode the directors column and drop nan rows
imdb_title_crew = imdb_title_crew.explode("directors")
imdb_title_crew.dropna(inplace=True)

## imdb.name.basics.csv.bz2

In [17]:
# load and display our data
imdb_name_basics = import_and_display("../data/raw/imdb.name.basics.csv.bz2")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606648 entries, 0 to 606647
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   nconst              606648 non-null  object 
 1   primary_name        606648 non-null  object 
 2   birth_year          82736 non-null   float64
 3   death_year          6783 non-null    float64
 4   primary_profession  555308 non-null  object 
 5   known_for_titles    576444 non-null  object 
dtypes: float64(2), object(4)
memory usage: 27.8+ MB


,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer","tt1470654,tt0363631,tt0104030,tt0102898"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department","tt0114371,tt2004304,tt1618448,tt1224387"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator","tt0452644,tt0452692,tt3458030,tt2178256"


We'll drop all columns except for nconst and primary_name. We could keep just those who have director in their primary profession, but we will be joining later, so it would be wasteful.

In [18]:
# keep only nconst and primary name columns
imdb_name_basics = imdb_name_basics[["nconst", "primary_name"]]